# TITLE CELL

OVERALL, EXPLAIN CODE SNIPPETS, have them interact with the rest of the algorithms, and explain the overall approach.

## The environment

Description of the environment, the task, reward, the MARL setting...   


The problem illustrated in this notebook is a Multi-Agent Reinforcement Learning (MARL) consisting of 2 players playing one against the other at a game called Connect 4. Connect 4 is a two-player game. The goal is to get 4 tokens aligned horizontally, vertically or diagonaly. The token are placed turn by turn and fall vertically until the bottom or another token. If the grid is full and nobody won, the game is a draw. The grid is a rectangle 7 cells wide and 6 cells high. The observation space is a 6*7 matrix filled with 1 if the agent has token on this cell and 0 if either nobody or the other agent has a token on this cell. The action space is an interger between 0 and 6 included that indicates on each column to drop a token.  

The reward is +1 for a win, 0 for a draw and -1 for a lose or for an illegal move. 




## 1. First Ingredient: Tree Search

### 1.1 Deterministic Tree Search: from BFS to Alpha-Beta Pruning


Given a certain state of the grid, we can compute all the possible positions with each action we can take. Then we can do the same for each action of the oppenent. This way, we can build a tree with each node representing the state of the board and the edges an action taken. A naive solution would be to explore all the tree until a certain depth. This algorithm thus works by taking the This method called Breadth-First Search (BFS) is simple to implement but is both memory and computation expensive as the number of nodes is multiplied by 7 at each depth. If we compute the tree in depth instead of 

#### 1.1.A The Minimax Algorithm, a version of DFS

The goal of the Minimax Algorithm is to find the best move for the current player in a zero-sum **deterministic** game. The current player is the maximizer and tries to maximize its own score/reward and the other player is the minimizer and tries to minimize the score/reward of the maximizer (thus maximizing its own score/reward). Each state (node) in the tree is fully expanded until a leaf node (terminal state) is reached. The value of an action is computed by the minimax formula:

$$
V(s) = \begin{cases}
R(s) & \text{if } s \text{ is a terminal state} \\

\max_{a \in \mathcal{A}(s)} V(\text{result}(s, a)) & \text{if } \text{player}(s) = \text{maximizer} \\
\min_{a \in \mathcal{A}(s)} V(\text{result}(s, a)) & \text{if } \text{player}(s) = \text{minimizer} \\
\end{cases}
$$

Where $\mathcal{A}(s)$ is the set of legal actions that can be taken from state $s$, $\text{result}(s, a)$ is the state resulting from taking action $a$ in state $s$, $\text{player}(s)$ is the player that has to play in state $s$ and $R(s)$ is the reward of state $s$. All of these functions are clearly defined in a fully observable deterministic game like Connect 4, Chess, Tic-Tac-Toe, Go, etc...

To compute the value of each state, we have to use a Depth-First Search (DFS) algorithm and backpropagate the value of leaf nodes to the root node, which is the current state of the game (although technically, minimax only needs to traverse the full tree from the starting state once and the game is "solved" for perfect play). However, this algorithm is not efficient as it explores the full tree and is thus not scalable to games with a high branching factor and/or depth. It also would not work in partially observable or stochastic environments. 

#### 1.1.B Alpha-Beta Pruning, improving the Minimax Algorithm and the use of heuristics.

Alpha-Beta Pruning is a way to improve the Minimax Algorithm by pruning branches that are guaranteed to be worse than the current best move. This is done by keeping track of two values, alpha and beta, which represent the minimum score the maximizer is assured of and the maximum score the minimizer is assured of respectively. When the algorithm finds a move that is better than the current alpha or beta, it updates the value and prunes the other branches and when the move under consideration is worse (for the current player) than the current alpha or beta, it prunes the branch. This way, the algorithm is able to expand a smaller number of nodes and thus is stricly more efficient than Minimax. The gap of performance between Minimax and Alpha-Beta Pruning depends a lot on the ordering of moves/actions to take when expanding a node. Considering a suboptimal move first will not allow the algorithm to prune as many branches as if it had considered the optimal move or a close-to-optimal move first.

Because even Alpha-Beta Pruning can be very computationally expensive, most modern game engines like Stockfish (for Chess) in high branching factor cannot explore the full tree until leaf nodes, even with pruning. Instead of relying on game results/rewards at the leaf node, game engines use heuristics (or even function approximations) to estimate the value of a state. In turn, the pruning is done not when an actual reward is "received" in a branch, but when the heuristics that are computed are worse than the current alpha or beta. Heuristics are also used for move ordering, as a good ordering can lead to more pruning and thus a faster search.

For example, in chess, heuristics can be the number and value of pieced on the board, a passed pawn, the control of the center, a knight's outpost (or conversely, a knight that is trapped on the edge of the board), etc... In Connect 4, a heuristic could be the number of 3-in-a-row positions, the number of 2-in-a-row positions and whether they are "open" on either end. Rules for these heuristics are often hand-crafted by experts in the game (similarly to hand-crafting features in the early days of function approximations in ML/RL) and then these heuristics are combined into a single value that represents the value of the state, either with hand-crafted weights or with some form of linear regression. This was the base of the pruning method used in Stockfish (in conjunction with alpha-beta search) until recent versions. 

In 2019/2020, Neural Networks were added to Stockfish to improve the evaluation heurisitcs, moving it closer to AlphaZero's approach (but with a very different search algorithm and training method). 


### 1.2 Monte-Carlo Tree Search

Mention decision time planning

#### 1.2.A Naive MCTS

Describe MCTS, link it to UCB = UCT

MCTS = probabilistic prunning + other stuff

#### 1.2.B MCTS with heuristics for rollouts or value estimation

Adding heuristics to MCTS.

PUCT

rollouts just for the end (nodes that we do not wish to expand further)

## Second Ingredient: Function Approximation

### Use of a value network and policy network in MCTS: the first step in AlphaGo

Show that a poorly trained value network/policy network are improved by MCTS :
- show that value estimates are improved by MCTS
- show that policy estimates are improved by MCTS (compared to depth 0)

Why MCTS and not alpha-beta pruning? --> avoid accumulation of errors.

### How to train these networks (for the first step in AlphaGo)

## Third Ingredient: Self-Play

The third ingredient of AlphaZero is self-play, which is a form of **background planning**. This technique was already used in algorithms like TD-Gammon, and AlphaGo, albeit in two different ways. Specifically, the policy of AlphaGo was first trained on human data, and then improved through self-play against the best policy among every previous iterations. AlphaGo Zero, AlphaZero and every subsequent algorithm using the same architecture, however, only use self-play to train the policy and value networks.


### How to generate training data

During training of Alpha(Go) Zero, the agent continuously plays games against itself using MCTS, the same algorithm used during the decision time, but here used for background planning. Using the current policy-value network pair to guide the search, the agent plays games until the end. It stores, in each visited state (state that was actually reached during the game), the final search probabilities for that state $\pi_\mathrm{PUCT}$. In the end, a reward (1 for a win, 0 for a draw, -1 for a loss) is assigned to the whole game trajectory. 

For Alpha(Go) Zero, this training data is generated asynchronously by multiple instances of the agent playing games against itself (and with multiple threads per instance for MCTS), and stored in a replay buffer, while the network is trained on mini-batches of this data in parallel. The replay buffer here plays a slightly different role than in DQN training, as it's role is not to stabilize the training by breaking the temporal correlation between samples and collecting more diverse data, but rather just to store the asynchronous data.

### How to train the networks: AlphaGo vs Alpha(Go) Zero

There are important differences in how the networks are trained with self play in AlphaGo and Alpha(Go) Zero, apart from the fact that AlphaGo first used human data to train the policy network. 

AlphaGo's self play improvement was done using a vanilla policy gradient algorithm (REINFORCE) without any baseline (so without using the value network). In fact, the value network was only trained at the end of the policy improvement phase, using just self-play games of the final agent. This value network was then used, in conjunction with the fast rollout policy to improve the search in MCTS-PUCT.

In Alpha(Go) Zero, the training objective for the policy is different. Instead of a policy gradient, or an advanced version like Actor Critic (making use of the value network being trained alongside the policy network), the policy is trained to match the final search probabilities $\pi_\mathrm{PUCT}$ obtained from MCTS-PUCT in every state. This is done by minimizing the KL divergence/cross entropy. There are several advantages of this approach. First, it is more stable and less noisy than policy gradients (because the full vector of probabilities is used, instead of just the value of the actions being taken). Second, it is more sample efficient, as information from states that ended up being not chosen is still used to train the policy. This is better both for other potential good moves that were not chosen, and for quickly pushing down the probabilities of bad moves that could have been chosen (but weren't). In effect, Alpha(Go) Zero's policy training continuously distills the search knowledge into the policy network, making it more and more accurate over time. The final policy, even without search (depth 0), is often very strong. In our Connect 4 example, it would probably beat you. [CODE]

In [1]:
# TODO add example

In both AlphaGo and Alpha(Go) Zero, the value network is trained using the final outcome of the game, without considering Q-values that were computed during the game by MCTS-PUCT (which would have been a form of bootstrapping/temporal difference learning). In zero-sum games with reward $\pm 1$, this is equivalent to training the value network to predict the probability of winning from a given state.

To conclude, Alpha Zero's training can be seen as on-policy, with some nuances. The value network is definitely trained on-policy, as it is trained on the final outcome of the game and nothing else. As for the policy network, one could argue for both cases (on and off-policy). Technically, the target policy that the network is trained to match is the search probabilities $\pi_\mathrm{PUCT}$, which are obtained from the current policy-value network pair and MCTS, not just the raw policy network. However, this policy network is not supposed to be used on its own, but rather in conjunction with MCTS, so one would argue that the combination (policy network + MCTS) is trained on-policy.

A small caveat to all of that is the use of the replay buffer. However, the replay buffer is quite shallow and provides only data from very recent games. It is also used to make the training asynchronous between network improvements and self-play rollout generation. We would tag Alpha Zero's training as *mildly off-policy*.

## Result! AlphaZero

### Describe AlphaZero
decition time planning with MCTS, background planning with self-play, and the use of deep neural networks.

Show a complete training

### Variants (MuZero, Gumbel MuZero, other like Efficient Zero?)

## Other experiments